Read dataset

In [1]:
import sys, os
import re
import pandas as pd
import itertools, nltk, string 

def read_file(file):
    f = open(file, 'r')

    pattern_title = '\[t\]'
    pattern_sentence = '(?<=##).+'
    pattern_aspect = '.+(?=##)'
#     pattern_aspect_anomali = '\[(u|p|s|cc|cs)\]'

    review = []
    for a in f:
        if re.search('##', a):
            sentence = re.findall(pattern_sentence, a)[0]
            aspect = re.findall(pattern_aspect, a)
#             anomali = re.findall(pattern_aspect_anomali, a)
#             if anomali:
#                 print(sentence, aspect)
#             else:
            flag = True
            if len(aspect) > 0:
                aspect = aspect[0]
                
#                 a_array = []
#                 tes = []
#                 tes = aspect.split(', ')
#                 for x in tes:
#                     a_array.append(x.split('[')[0])
#                 for a in a_array:
#                     if a not in sentence:
#                         flag = False
            else:
                aspect = ''
            if flag:
                review.append((sentence, aspect))

    df = pd.DataFrame(columns=['review','target'])
    for r in review:
        df = df.append({'review': r[0], 'target': r[1]}, ignore_index=True)
        
    return df
        
positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))
    
        
read_lexicon()
df = read_file('dataset/bing_liu/Nokia 6610.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Farza Nurifan\\Documents\\OneDrive\\Dokumen\\Code Paper\\Tugas Akhir\\GAK GUNAAAAAAA\\opinion-lexicon-English\\positive-words.txt'

In [2]:
df.head()

,review,target
0,i am a business user who heavily depend on mob...,
1,there is much which has been said in other rev...,"phone[+3], work[+2]"
2,just double check with customer service to ens...,
3,after several years of torture in the hands of...,at&t customer service[-2]
4,"i have had the phone for 1 week , the signal q...",signal quality[+3]


In [3]:
from nltk.parse.corenlp import CoreNLPDependencyParser

parser = CoreNLPDependencyParser()
parse = next(parser.raw_parse("my name is khan"))

In [4]:
import os
os.environ["CORENLP_HOME"] = r'C:\stanford-corenlp-full-2018-10-05'

import corenlp 
client = corenlp.CoreNLPClient()

def preprocess(sentences, chunk = False):
    res = []
    for sentence in sentences:
        text = sentence.lower()
        if chunk:
            try:
                pattern = '[{pos:NN}][{pos:NN}][{pos:NN}] | [{pos:NN}][{pos:NN}]'
                matches = client.tokensregex(sentence, pattern)
                response = matches['sentences'][0]
                len_chunk = response['length']
                if len_chunk > 0:
                    for index in range(0, len_chunk):
                        replacer = response[str(index)]['text']
                        text = text.replace(replacer, '-'.join(replacer.split(' ')))
            except:
                text = text
        res.append(text)
    return res

In [5]:
import requests
import re

def get_tregex(text):
    url = "http://localhost:9001/tregex"
    request_params = {"pattern": "S < (NP $ VP)"}
    r = requests.post(url, data=text, params=request_params, timeout=120)
    print(r)
    try:
        return r.json()['sentences'][0]
    except:
        return []

def sentence_from_tree(s):
    p_wh = r'(?<=WHADVP).*?(?=\))'
    p_wh2 = r'(?<=WHNP).*?(?=\))'
    pattern = r'(?<= )[a-zA-Z].*?(?=\))'
    replaced = s.replace('\r\n', '')
    wh = re.findall(p_wh, replaced)
    wh2 = re.findall(p_wh2, replaced)
    for x in wh:
        replaced = replaced.replace(x, '')
    for x in wh2:
        replaced = replaced.replace(x, '')
    res = ' '.join(re.findall(pattern, replaced))
    return res

def get_clauses(sentences):
    clauses = []
    res = get_tregex(sentences)
    length = len(res)
    for x in range(0, length):
        clauses.append(sentence_from_tree(res[str(x)]['match']))
    if length > 1:
        for x in range(0, length - 1):
            clauses[x] = clauses[x].replace(clauses[x+1], '')
    return clauses

In [6]:
# adding new dependencies for DP 'nmod' and'advmod'
dep_DP = ['amod', 'prep', 'nsubj', 'csubj', 'xsubj', 'dobj', 'iobj'] #advmod, dep, cop, mark, nsubjpass, pobj, acomp, xcomp, csubjpass, and poss 
conj_DP = ['conj']

In [7]:
NN = ['NN', 'NNS', 'NNP', 'NNPS']
JJ = ['JJ', 'JJR', 'JJS']

def rule_1_1(parse, f, o_expanded):
    #Rule 1.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in dep_DP):
            # Rule 1.1
            if(w1[0] in o_expanded):
                if w2[1] in NN and w2[0] not in f:
                    return True, w2[0], w1[0]
            elif(w2[0] in o_expanded):
                if w1[1] in NN and w1[0] not in f:
                    return True, w1[0], w2[0]
    return False, None, None

def rule_1_2(parse, f, o_expanded):
    # Rule 1.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP):
            H = ''
            O = ''
            if w1[0] in o_expanded:
                H = w2[0]
                O = w1
            elif w2[0] in o_expanded:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] in NN and w2[0] not in f:
                            return True, w2[0], O[0]
                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] in NN and w1[0] not in f:
                            return True, w1[0], O[0]
    return False, None, None

def rule_4_1(parse, f, o_expanded):
    # Rule 4.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in conj_DP):
            if w1[0] in o_expanded:
                if w2[1] in JJ and w2[0] not in o_expanded:
                    return True, w2[0]

            elif w2[0] in o_expanded:
                if w1[1] in JJ and w1[0] not in o_expanded:
                    return True, w1[0]

    return False, None

def rule_4_2(parse, f, o_expanded):
    # Rule 4.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP or dep in conj_DP):
            H = ''
            O = ''
            if w1[0] in o_expanded:
                H = w2[0]
                O = w1
            elif w2[0] in o_expanded:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] in JJ and w2[0] not in o_expanded:
                            return True, w2[0]

                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] in JJ and w1[0] not in o_expanded:
                            return True, w1[0]
    
    return False, None

def rule_3_1(parse, f):
    #Rule 3.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in conj_DP):
            if(w1[0] in f): 
                if w2[1] in NN and w2[0] not in f:
                    return True, w2[0], w1[0]
            elif(w2[0] in f):          
                if w1[1] in NN and w1[0] not in f:
                    return True, w1[0], w2[0]
                
    return False, None, None

def rule_3_2(parse, f):
    # Rule 3.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP or dep in conj_DP):
            H = ''
            O = ''
            if w1[0] in f:
                H = w2[0]
                O = w1
            elif w2[0] in f:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] in NN and w2[0] not in f:
                            return True, w2[0], O[0]
                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] in NN and w1[0] not in f:
                            return True, w1[0], O[0]

    return False, None, None

def rule_2_1(parse, f, o_expanded):
    # Rule 2.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in dep_DP):
            if w1[0] in f:
                if w2[1] in JJ and w2[0] not in o_expanded:
                    return True, w2[0]

            elif w2[0] in f:
                if w1[1] in JJ and w1[0] not in o_expanded:
                    return True, w1[0]
                
    return False, None

def rule_2_2(parse, f, o_expanded):      
    # Rule 2.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP):
            H = ''
            O = ''
            if w1[0] in f:
                H = w2[0]
                O = w1
            elif w2[0] in f:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] in JJ and w2[0] not in o_expanded:    
                            return True, w2[0]
                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] in JJ and w1[0] not in o_expanded:
                            return True, w1[0]

    return False, None

def clause(other_clause, temp, parse, f):
    res = []
    if other_clause and len(temp) == 0:
        for (w1, dep, w2) in list(parse.triples()):
            if w1[1] in NN and w1[0] not in f:
                res.append(w1[0])
    if len(res) > 0:
        return True, res
    return False, None

In [8]:
import pandas as pd
candidate_aspect = []
new_opinion = []
op_set = positive_lexicon + negative_lexicon

def double_propagation(O: op_set, reviews, targets, output_file, save_to_file = False):
    o_expanded = O
    f = []
    is_stop = False
    flag_cycle = 0
    
    t_a_p = []
    pair_a_o = []
    a_p = []
    r_p = []
    
    while (not is_stop):
        f_i = []
        o_i = []
    
        index = 0

        for sentence in reviews:
            aspect_sentence = []
            if flag_cycle:
                aspect_sentence = t_a_p[index].split('|')
                
            temp = []
            pair = []
                
            try:
                parse = next(parser.raw_parse(sentence))
            except:
                continue

            # Rule 1.1
            is_true, t_a, o = rule_1_1(parse, aspect_sentence, o_expanded)
            if is_true:
                f_i.append(t_a)
                temp.append(t_a)
                pair.append(t_a + '!' + o)

            # Rule 1.2
            is_true, t_a, o = rule_1_2(parse, aspect_sentence, o_expanded)
            if is_true:
                f_i.append(t_a)
                temp.append(t_a)
                pair.append(t_a + '!' + o)

            # Rule 4.1
            is_true, t_o = rule_4_1(parse, f, o_expanded)
            if is_true:
                o_i.append(t_o)

            # Rule 4.2
            is_true, t_o = rule_4_2(parse, f, o_expanded)
            if is_true:
                o_i.append(t_o)

                                    
            if flag_cycle == 0:
                r_p.append(sentence)
                a_array = []
                tes = []
                try:
                    tes = targets[index].split(', ')
                except:
                    tes = []
                for x in tes:
                    splitted = x.split('[')
                    if len(splitted) > 1:
                        a_array.append(splitted[0] + '!' + splitted[1][0])
                    else:
                        a_array.append(splitted[0])

                a_p.append('|'.join(a_array))
                t_a_p.append('|'.join(temp))
                pair_a_o.append('|'.join(pair))
            else:
                if len(temp) != 0:
                    t_a_p[index] += '|' + '|'.join(temp)
                    pair_a_o[index] += '|' + '|'.join(pair)
            index += 1
            

        #calculate target and opinion expanded
        f = f + f_i 
        o_expanded = o_expanded + o_i

        
        #reread review, and run rule 3.1, 3.2, 2.1, and 2.2
        index = 0
        f_ii = []
        o_ii = []
        
        for sentence in reviews:
            
            aspect_sentence = []
            if flag_cycle:
                aspect_sentence = t_a_p[index].split('|')
                
            temp = []
            pair = []
                
            try:
                parse = next(parser.raw_parse(sentence))
            except:
                continue

            # Rule 3.1
            is_true, t_a, o = rule_3_1(parse, aspect_sentence)
            if is_true:
                f_ii.append(t_a)
                temp.append(t_a)
                pair.append(t_a + '!' + o)

            # Rule 3.2
            is_true, t_a, o = rule_3_2(parse, aspect_sentence)
            if is_true:
                f_ii.append(t_a)
                temp.append(t_a)
                pair.append(t_a + '!' + o)


            # Rule 2.1
            is_true, t_o = rule_2_1(parse, f, o_expanded)
            if is_true:
                o_ii.append(t_o)

            # Rule 2.2
            is_true, t_o = rule_2_2(parse, f, o_expanded)
            if is_true:
                o_ii.append(t_o)

            if len(temp) != 0:
                t_a_p[index] += '|' + '|'.join(temp)
                pair_a_o[index] += '|' + '|'.join(pair)

            index += 1
            
        f_i = f_i + f_ii
        o_i = o_i + o_ii
        f = f + f_ii
        o_expanded = o_expanded + o_ii     
        
        flag_cycle = 1
        
        if(len(f_i) == 0 and len(o_i) == 0):
            if save_to_file == True:
                out = pd.DataFrame(r_p)
                out['aspect'] = a_p
                out['prediction'] = pair_a_o
                out.to_csv(output_file)
            is_stop = True
        
    return f, o_expanded

In [9]:
dp_aspect, opinion_expand = double_propagation(op_set, preprocess(df['review'], False), df['target'], 'coba.csv', True)

In [10]:
def calculate_frequency(aspects):
    aspect_frequency = {}
    
    for aspect in aspects:
        if(aspect in aspect_frequency):
            aspect_frequency[aspect] += 1
        else:
            aspect_frequency[aspect] = 1
            
    return aspect_frequency

k_DP = ['conj', 'compound']
def pruning_based_on_clause(aspect_frequency, reviews, predictions):
    pruning = []
    index = 0
    for review in reviews:
        temp = predictions[index]
        index += 1
        if isinstance(temp, str):
            prediction = temp.split('|')
        else:
            continue
        parse = next(parser.raw_parse(review))
        for (w1, dep, w2) in list(parse.triples()):
            if(w1[0] in aspect_frequency and w2[0] in aspect_frequency):
                if(dep not in k_DP):
                    if(aspect_frequency[w1[0]] > aspect_frequency[w2[0]]):
                        if w2[0] in prediction:
                            pruning.append(w2[0])
                    elif(aspect_frequency[w1[0]] < aspect_frequency[w2[0]]):
                        if w1[0] in prediction:
                            pruning.append(w1[0])
                
    return pruning
def pruning_based_other_products_and_dealers(aspect_frequency, reviews,predictions, window=3):
    pruning = []
    ProductINDI = ["compare to", "compare with", "better than", "worse than"]
    DealerINDI  = ["shop with", "buy from"]
    count = 0
    for review in reviews:
        temp = predictions[count]
        count += 1
        if isinstance(temp, str):
            prediction = temp.split('|')
        else:
            continue
        if any(indication in review for indication in ProductINDI):
            tokens = nltk.word_tokenize(review)
            index = 0
            while index < len(tokens) - 1:
                if tokens[index] + " " + tokens[index + 1] in ProductINDI:
                    index += 2
                    for x in range(window):
                        next_window = index + x + 1;
                        if next_window < len(tokens) and tokens[next_window] in aspect_frequency:
                            if tokens[next_window] in prediction:
                                pruning.append(tokens[next_window])
                else :
                    index += 1
                    
        if any(indication in review for indication in DealerINDI):
            tokens = nltk.word_tokenize(review)
            index = 0
            while index < len(tokens) - 1:
                if tokens[index] + " " + tokens[index + 1] in DealerINDI:
                    index += 2
                    for x in range(window):
                        next_window = index + x + 1;
                        if next_window < len(tokens) and tokens[next_window] in aspect_frequency:
                            if tokens[next_window] in prediction:
                                pruning.append(tokens[next_window])
                else :
                    index += 1 
                        
    return pruning


In [11]:
import pandas as pd

hs = pd.read_csv("coba.csv")

import re
import math
import numpy as np
target = []

for t in hs['aspect']:
    if t is not np.nan:
        for s in t.split('|'):
            target.append(s.split('!')[0])
            
tp = calculate_frequency(target)
ap = calculate_frequency(dp_aspect)
pruning_clause = pruning_based_on_clause(ap,hs['0'], hs['prediction'])
prun_dealer_product = pruning_based_other_products_and_dealers(ap, hs['0'], hs['prediction'])

a = pruning_clause + prun_dealer_product

for x in list(dict.fromkeys(dp_aspect)):
    flag = True
    splitted = x.split('-')
    for word in splitted:
        for match in list(dict.fromkeys(dp_aspect)):
            if word in match and word != match:
                flag = False
                
    if ap[x] == 1 and flag:
        ap[x] -= 1

for freq in list(dict.fromkeys(a)):
    try:
        if ap[freq] > 0:
            ap[freq] -= 1
    except: continue

new_dp_aspect = []
for a in ap:
    if ap[a] > 0:
        new_dp_aspect.append(a)

In [12]:
def calculate_performance():
    atas_recall = 0
    atas_precision = 0
    bawah_precision = 0
    bawah_recall = 0

    new_target = []
    for x in list(dict.fromkeys(target)):
        for y in x.split(' '):
            new_target.append(y.split('!')[0])

    new_aspect = []
    for x in list(dict.fromkeys(new_dp_aspect)):
        for y in x.split('-'):
            new_aspect.append(y.split('!')[0])

    for x in tp:
        bawah_precision += tp[x]
        for y in x.split(' '):
            flag = False
            for apt in new_aspect:
                if y in apt:
                    atas_precision += tp[x]
                    flag = True
                    break
            if flag:
                break

    for x in ap:
        bawah_recall += ap[x]
    #     print(x.split('-'))
        for y in x.split('-'):
            flag = False
            for tgt in new_target:
                if y in tgt:
        #             print(y)
                    atas_recall += ap[x]
                    flag = True
                    break
            if flag:
                break

    precision = atas_precision/bawah_precision
    recall = atas_recall/bawah_recall
    f1 = (2 * precision * recall) / (precision + recall)
    
    return precision, recall, f1

In [13]:
calculate_performance()

(0.8041958041958042, 0.7617924528301887, 0.7824200360193366)

In [14]:
calculate_performance()

(0.8041958041958042, 0.7617924528301887, 0.7824200360193366)

In [15]:
df = pd.read_csv('sentence_type.csv')

In [47]:
sentences = []
targets = []
count = 0
for i in range(0, len(df)):
    if df['type'][i] == 'compound_complex_sentence':
#     if df['type'][i] == 'phrase' or df['type'][i] == 'simple_sentence':
        sentences.append(df['review'][i])
        targets.append(df['aspect'][i])